# Tabular Playground Series Sep 2021

In [1]:
# Installing the modules

!pip3 install kaggle

In [26]:
# Required modules

import numpy as np
import pandas as pd
import tensorflow as tf

from zipfile import ZipFile
from google.colab import files
from matplotlib import pyplot as plt

from sklearn.impute import SimpleImputer

In [3]:
# Config

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)

In [4]:
# Upload the creds

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anurag1817","key":"bb26996d6a18a21a05fb9a60176c33dd"}'}

In [5]:
# Create kaggle folder

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
# Test the command

!kaggle competitions download -c tabular-playground-series-sep-2021

  0% 0.00/1.10M [00:00<?, ?B/s]
100% 1.10M/1.10M [00:00<00:00, 76.3MB/s]
 95% 193M/203M [00:02<00:00, 82.1MB/s]
100% 203M/203M [00:02<00:00, 103MB/s] 
 96% 379M/394M [00:03<00:00, 128MB/s]
100% 394M/394M [00:03<00:00, 109MB/s]


In [8]:
# Extract the zip file

with ZipFile('/content/train.csv.zip', 'r') as zf:
    zf.extractall('./')

with ZipFile('/content/test.csv.zip', 'r') as zf:
    zf.extractall('./')

with ZipFile('/content/sample_solution.csv.zip', 'r') as zf:
    zf.extractall('./')

In [10]:
# Load the data

train = pd.read_csv('train.csv', index_col='id')
train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,0.59881,1.423200e+09,0.27240,9.455600,-0.050305,1938.300,8.6331,4.0607,26.8670,-1.180,10961.00,1.5397,135.3200,-1.49650,440.080,2.590100e+12,2.194200e+09,2968800.0,0.001431,13.3270,0.75050,18509.0,146820.0,-0.000276,1.090600e+16,1705.400,414.29,3.5392,1888.0,0.968930,...,-0.001081,6.1244,1.231800e+11,275.9200,5308500.0,1704.000,5.022400e+10,53.3980,-2.2012,6871.0,3.8862,-0.00558,5252.100,166.690,1.60740,0.66534,7768.900,0.99662,1.125700e+11,2.2432,0.934160,0.65056,94569.0,21.471,8214.100,0.288010,0.097826,0.001071,1.412400e+09,0.11093,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,1.20930,3.334100e+09,0.28631,-0.012858,-0.019912,10.284,6.1872,1.0419,4.6404,31.877,123620.00,1.3951,125.8100,1.19890,136.450,9.098100e+09,4.004100e+10,1564000.0,0.000204,3.1074,1.50330,238000.0,21440.0,-0.001344,3.079400e+16,229.100,844.82,1.4680,4726.5,0.915380,...,0.254100,6.9191,1.832400e+11,9.6510,32800.0,1480.600,2.300600e+10,44.0510,205.6900,4295.3,13.3880,0.46843,754.610,83.233,1.18900,29.55000,7343.700,0.99815,4.877700e+13,1.2708,-0.000969,5.29520,6779.0,227.720,34.342,0.340300,0.143370,0.049276,1.903200e+09,0.97673,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,0.38164,1.230300e+09,0.25807,2.455600,NaN,26.873,7.5463,1.9967,1.9526,817.760,-2948.70,2.0054,1.6826,1.19680,74.624,-3.273900e+10,5.718900e+10,11058.0,-0.003097,8.0241,1.13180,27940.0,862460.0,-0.002207,5.849100e+13,-897.840,NaN,1.3561,3063.4,0.086232,...,0.260260,6.1052,1.013300e+11,357.2700,1476600.0,90.845,1.306200e+09,2.3731,391.3700,2965.3,NaN,0.49459,43.524,138.520,1.10790,0.91948,47.915,NaN,1.510500e+12,3.4663,0.560950,4.13090,95531.0,39.486,-83.148,0.084881,0.032222,0.001668,1.436500e+07,0.20102,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,1.10960,1.223100e+09,0.30944,10.370000,-0.106260,533.840,7.8490,1.0379,8.0030,12.349,-195.28,2.5598,92.1420,0.63789,1054.900,-1.204100e+10,5.187300e+12,1475400.0,1.036500,1.1903,0.98941,301200.0,NaN,-0.000007,-9.299200e+13,-10.818,1020.30,2.9553,3342.5,-0.000372,...,0.372830,1.5606,1.835400e+10,-3.4298,6485700.0,2120.000,3.081200e+10,34.0560,157.4300,3724.5,8.4211,0.40778,2971.200,204.700,-0.97998,9.94050,12011.000,0.99898,5.063400e+13,1.2261,0.250200,0.72974,373690.0,194.650,120.930,0.260710,0.234240,-0.002794,1.442300e+09,-0.01182,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,1.05310,2.634100e+09,0.29782,2.654800,NaN,1808.900,7.2783,3.9757,NaN,NaN,29520.00,3.4225,96.7250,0.79725,215.570,1.732600e+13,2.635200e+12,2161200.0,0.895470,6.8257,0.97413,142620.0,231350.0,0.001257,1.012500e+16,51.508,293.76,1.3351,3042.1,0.006791,...,0.085690,1.5846,3.825200e+10,130.7000,102100.0,1951.800,1.142800e+10,58.5660,176.8300,1279.0,4.9662,0.47912,-70.278,10.887,1.14340,6.19120,197.470,NaN,1.574800e+13,1.0083,0.339530,13.48600,201300.0,38.842,324.000,0.238250,0.141550,0.002208,5.830700e+09,0.92739,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [11]:
# Inspecting the data

train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 957919 entries, 0 to 957918
Columns: 119 entries, f1 to claim
dtypes: float64(118), int64(1)
memory usage: 877.0 MB


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,942672.000000,942729.000000,942428.000000,942359.000000,942514.000000,942398.000000,942415.000000,9.425460e+05,9.426700e+05,942696.000000,942494.000000,9.423260e+05,942455.000000,942697.000000,942410.000000,942475.000000,942492.000000,942594.000000,942445.000000,942464.000000,942465.000000,942641.000000,942563.000000,942289.000000,942413.000000,9.425610e+05,9.424750e+05,9.426540e+05,942504.000000,942527.000000,942241.000000,9.423900e+05,9.424270e+05,942671.000000,9.425850e+05,942556.000000,942607.000000,942485.000000,942360.000000,942551.000000,...,942599.000000,942573.000000,9.424340e+05,942292.000000,9.425340e+05,942470.000000,9.423960e+05,942603.000000,942371.000000,942474.000000,942437.000000,942412.000000,942427.000000,942462.000000,942505.000000,942320.000000,942634.000000,942654.000000,9.426310e+05,942485.000000,942393.000000,942570.000000,9.427510e+05,942319.000000,942721.000000,942533.000000,942375.000000,942535.000000,9.425790e+05,942390.000000,942554.000000,942420.000000,942509.000000,942686.000000,942481.000000,942360.000000,9.423300e+05,942512.000000,942707.000000,957919.000000
mean,0.090201,0.345964,4068.744207,0.201214,0.304869,-0.071458,1620.843815,3.771642e+05,1.806054e+15,5323.442367,0.729518,1.844436e+09,0.247759,6.999152,0.019394,444.804439,6.892520,4.491982,22.448660,203.782031,61070.567499,2.269597,87.166891,0.341240,414.983986,3.379988e+12,1.254164e+12,2.256887e+06,0.329054,7.882455,0.394211,1.344700e+05,3.579055e+05,-0.000005,2.782318e+16,185.485192,406.072571,1.768743,1981.074344,0.359381,...,0.194237,3.240370,1.053811e+11,152.850973,6.128694e+06,635.301741,3.251414e+10,26.598567,207.260994,3805.100719,6.733556,0.366754,4868.608259,132.264261,0.821190,13.120677,3849.687894,0.999970,1.415684e+13,1.683151,0.425840,20.213776,3.215760e+05,548.647493,3855.616303,0.178027,0.160832,0.014196,1.671873e+09,0.398622,-19.926398,2.074530,23.885245,1.748777,63152.973540,1.208876,4.276905e+16,3959.204669,0.559267,0.498492
std,0.043564,0.146251,6415.829440,0.212510,0.145343,2.123777,1276.281403,3.454325e+05,2.335204e+15,10068.380032,1.495535,2.125478e+09,0.101156,6.621472,0.101880,631.327086,1.715735,3.899241,14.609968,281.015052,89859.616863,0.896001,37.350264,1.643595,314.822410,5.655524e+12,1.642736e+12,2.303909e+06,0.433861,5.939864,0.781932,2.036605e+05,4.626670e+05,0.001534,3.452707e+16,701.769083,314.708639,0.588984,1958.688237,0.441752,...,0.162364,1.992068,9.896094e+10,227.888751,8.767985e+06,583.468704,3.069150e+10,25.454605,158.215276,3532.626437,3.158678,0.146413,8428.424099,110.058170,0.712584,12.740238,6437.386650,0.001535,1.640385e+13,0.712016,0.283651,19.617660,3.277370e+05,864.175628,6670.256245,0.123319,0.141616,0.016259,1.875733e+09,0.298071,18.578439,0.895793,45.581360,10.088848,92435.016241,0.114959,6.732441e+16,3155.991777,0.408426,0.499998
min,-0.149910,-0.019044,-9421.700000,-0.082122,-0.006990,-12.791000,-224.800000,-2.984300e+04,-1.153300e+15,-26404.000000,-8.086300,-4.081000e+08,-0.103800,-0.853760,-0.335660,-116.880000,-3.664500,-0.066527,-4.422500,-58.834000,-84079.000000,-6.009400,-20.514000,-5.735200,-71.502000,-6.956700e+11,-9.384200e+11,-4.706000e+05,-0.005659,-0.529990,-3.813500,-3.496500e+05,-6.055900e+05,-0.003881,-2.068900e+16,-2414.300000,-40.881000,0.546100,-433.700000,-0.007641,...,-0.017615,0.964200,-7.345700e+10,-28.752000,-2.992000e+06,-74.545000,-5.949500e+09,-7.616400,-22.576000,-296.780000,-0.257570,-0.012238,-12829.000000,-12.922000,-3.293300,-1.352400,-7764.300000,0.996100,-5.714600e+12,0.608200,-0.034559,-4.294900,-2.277700e+05,-222.210000,-11581.000000,-0.029027,-0.066726,-0.007535,-5.877000e+08,-0.042355,-105.860000,0.277040,-27.69

In [24]:
# Correlation matrix

train.corr().unstack().sort_values(kind="quicksort").tolist()

[-0.05638748301540266,
 -0.05638748301540266,
 -0.025281667539417508,
 -0.025281667539417508,
 -0.02526813592933988,
 -0.02526813592933988,
 -0.02504067168585802,
 -0.02504067168585802,
 -0.02252580038521992,
 -0.02252580038521992,
 -0.02138194444756649,
 -0.02138194444756649,
 -0.02133231317916443,
 -0.02133231317916443,
 -0.020626842509669174,
 -0.020626842509669174,
 -0.02024091098465075,
 -0.02024091098465075,
 -0.01972561801317563,
 -0.01972561801317563,
 -0.01808506596638768,
 -0.01808506596638768,
 -0.018027540989181284,
 -0.018027540989181284,
 -0.017829201173323574,
 -0.017829201173323574,
 -0.017815071931463547,
 -0.017815071931463547,
 -0.01773974051627119,
 -0.01773974051627119,
 -0.017665757392557175,
 -0.017665757392557175,
 -0.017547599660950174,
 -0.017547599660950174,
 -0.016796877220875,
 -0.016796877220875,
 -0.0167608022920634,
 -0.0167608022920634,
 -0.01619277109084707,
 -0.01619277109084707,
 -0.016189338516879162,
 -0.016189338516879162,
 -0.016130982185654057,


In [12]:
# Load the data

test = pd.read_csv('test.csv', index_col='id')
test.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
957919,0.165850,0.487050,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,6221.000,1.17200,5.175000e+09,0.25831,3.42050,0.044983,1001.600,10.1040,10.5360,19.517,-1.1209,1577.60,2.18470,96.109,1.1535,95.865,2.847100e+12,9.642100e+10,6406000.0,0.002990,6.6353,-0.015913,21723.0,67604.0,-0.000227,6.046200e+16,203.580,301.790,1.3461,5524.60,-0.003163,...,NaN,0.510410,4.0322,2.477900e+11,69.2640,5494800.0,1239.00,5.349300e+08,5.3086,182.590,829.17,5.2319,0.010301,3640.800,112.120,-0.43442,11.16100,3755.700,0.99926,5.338200e+10,1.3867,-0.001878,58.7940,75657.0,187.240,1258.90,0.163340,0.055398,0.020002,7.956400e+08,0.16253,-22.1890,2.0655,0.430880,-10.7410,81606.0,1.1940,1.980400e+14,2017.1,0.46357
957920,0.129650,0.373480,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,934.430,1.89790,4.905100e+09,0.28471,2.13870,-0.034328,35.223,6.5873,12.7340,29.590,4.1639,31915.00,3.11580,109.430,-0.8522,991.370,4.700100e+09,1.618200e+12,967810.0,1.021500,2.9617,0.810310,415570.0,384720.0,0.002243,6.712900e+15,228.590,33.952,1.4938,2067.50,-0.002343,...,-5.3544,0.067286,5.4044,2.462600e+11,810.3000,860670.0,2165.20,2.711500e+08,10.2320,515.410,262.71,5.6357,0.374220,420.160,116.230,1.26980,0.78619,1207.000,1.00070,2.065000e+13,1.2888,0.248850,9.9857,16323.0,244.460,647.58,0.253820,0.008800,0.018479,1.120600e+08,0.81528,-1.6342,1.5736,-1.071200,11.8320,90114.0,1.1507,4.388000e+16,6638.9,0.28125
957921,0.120190,0.445210,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,131.810,-0.48404,3.800000e+06,0.37173,0.40245,0.015119,1991.600,6.0050,0.9919,11.949,888.0700,887.70,3.52370,108.140,-1.6542,528.140,2.463600e+12,9.005900e+09,7259.3,1.007400,2.6123,1.062500,-208010.0,65708.0,0.001391,8.129800e+13,-84.032,NaN,1.7556,4729.20,0.003527,...,-1.4805,0.064203,3.0197,1.684400e+11,212.8300,7891000.0,277.16,4.047900e+10,1.1170,393.510,8859.10,8.8098,0.405350,393.650,305.140,1.32660,30.85200,2803.700,0.99925,4.722600e+12,1.5003,0.493380,37.0470,139070.0,2464.000,4724.80,0.299160,0.093046,0.018516,6.519300e+07,0.81831,-32.7800,2.1364,-1.931200,-3.2804,37739.0,1.1548,1.718100e+14,5844.0,0.13797
957922,0.054008,0.395960,996.14,0.85934,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,-26.473,-2.12930,4.675400e+09,0.26659,1.09680,-0.083571,1987.500,6.8989,11.1230,23.164,133.5700,-815.32,2.72950,82.011,3.8374,304.570,-1.315900e+10,-8.324500e+09,2297700.0,0.099522,4.9249,0.659900,48773.0,199560.0,-0.000621,3.993800e+16,1400.400,503.660,1.9131,2811.60,0.002362,...,-2.3828,0.128580,1.4494,2.681000e+11,66.5990,-733900.0,843.42,1.216700e+10,3.6871,243.590,1199.70,4.5092,0.377030,-36.886,11.401,0.97205,23.64200,1882.700,0.99922,1.568000e+13,1.0456,0.548180,27.6650,813040.0,1332.000,359.17,0.026553,0.668620,-0.001260,1.995100e+09,0.86559,-2.4162,1.5199,-0.011633,1.3840,26849.0,1.1490,2.138800e+17,6173.3,0.32910
957923,0.079947,-0.006919,10574.00,0.34845,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,5999.400,1.87380,1.857300e+09,0.32259,2.21150,-0.107200,661.070,6.5391,6.8495,7.258,468.7400,10316.00,0.79638,111.520,1.4098,429.790,4.121700e+12,-3.945700e+08,7119.4,1.027700,16.1210,0.994510,372800.0,12042.0,-0.000659,1.869200e+14,-129.580,933.150,2.7392,490.84,0.012809,...,-3.2187,0.502290,6.9219,1.487300e+11,9.9686,-35362.0,854.84,2.603900e+10,13.9430,14.552,675.62,6.8424,0.376850,3452.100,35.434,0.94128,0.95142,-56.004,0.99969,2.009700e+13,1.2255,0.713930,14.1820,4362.6,12.007,10653.00,0.165300,0.227090,0.020485,-1.050400e+07,0.25190,-18.6300,3.7387,0.757080,-4.9405,50336.0,1.2488,2.151300e+17,2250.1,0.33796


Almost every columns has some missing values, in it so we can use the simpleimputer with `mean` to fill the gaps.

In [15]:
# Missing values

if train.isna().any().any():
    print(train.isna().sum()/train.shape[0])
else:
    print("No Missing values")

f1       0.015917
f2       0.015857
f3       0.016172
f4       0.016244
f5       0.016082
           ...   
f115     0.016243
f116     0.016274
f117     0.016084
f118     0.015880
claim    0.000000
Length: 119, dtype: float64


In [19]:
# Imputing the missing values with mean

for col in train.columns:
    if train[col].isna().any():
        imputer = SimpleImputer(strategy='mean')
        train[col] = imputer.fit_transform(train[col].values.reshape(-1, 1))
        test[col] = imputer.transform(test[col].values.reshape(-1, 1))

In [50]:
# Seperating out features and labels

X = train.drop(['claim'], axis=1)
y = train['claim']

## Model-1

In [56]:
# Model Definition

model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [57]:
# Compiling the model

optim = tf.keras.optimizers.Adam()
loss = tf.keras.losses.binary_crossentropy
metrics = tf.keras.metrics.AUC()

model1.compile(optimizer=optim, loss=loss, metrics=[metrics])

In [58]:
# Fitting the model

epochs = 20
batch_size = 32

history = model1.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size)

Epoch 1/20
23948/23948 [==============================] - 66s 3ms/step - loss: 6117123948544.0000 - auc_1: 0.4995 - val_loss: 0.6932 - val_auc_1: 0.5000
Epoch 2/20
23948/23948 [==============================] - 64s 3ms/step - loss: 1298063232.0000 - auc_1: 0.4995 - val_loss: 0.6932 - val_auc_1: 0.5000
Epoch 3/20
23948/23948 [==============================] - 67s 3ms/step - loss: 385103392.0000 - auc_1: 0.4988 - val_loss: 0.6932 - val_auc_1: 0.5000
Epoch 4/20
23948/23948 [==============================] - 65s 3ms/step - loss: 1603956480.0000 - auc_1: 0.4999 - val_loss: 0.6932 - val_auc_1: 0.5000
Epoch 5/20
23948/23948 [==============================] - 64s 3ms/step - loss: 178214624.0000 - auc_1: 0.4998 - val_loss: 0.6932 - val_auc_1: 0.5000
Epoch 6/20
23948/23948 [==============================] - 60s 3ms/step - loss: 627402752.0000 - auc_1: 0.4998 - val_loss: 0.6932 - val_auc_1: 0.5000
Epoch 7/20
23948/23948 [==============================] - 65s 3ms/step - loss: 0.6932 - auc_1: 0.499

In [59]:
# Model Prediction

preds = model1.predict(test)
preds

array([[0.49812272],
       [0.49812272],
       [0.49812272],
       ...,
       [0.49812272],
       [0.49812272],
       [0.49812272]], dtype=float32)

In [60]:
# Generating output file

submission = pd.read_csv('sample_solution.csv')
submission['claim'] = preds
submission.to_csv('output.csv', index=False)

In [61]:
# Submission

!kaggle competitions submit -c tabular-playground-series-sep-2021 -f output.csv -m "TF loss=mse metrics=auc without error"

100% 8.90M/8.90M [00:00<00:00, 20.1MB/s]
Successfully submitted to Tabular Playground Series - Sep 2021